
# GPU Numba and CuPy Parallelization of Matrix Multiplication 

Similary to the multicore parallelization lab, in this lab we will be using Numba and CuPy to accelerate matrix-matrix multiplications using GPU. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba for CUDA](https://numba.readthedocs.io/en/stable/cuda/index.html)
* [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb)
* [Numba.CUDA by Graham Markell](https://github.com/numba/nvidia-cuda-tutorial)
* [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/)
* [CuPy Basics](https://docs.cupy.dev/en/stable/user_guide/basic.html)

[//]: <> (GEOPHYS 257 Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

<br>

### Exercise 0

* You need to request a *T4* node on the cluster. Don't forget that you need to add **--gres=gpu** to your srun command.
* Reminder: on the *T4* nodes you need to load a different version of Python:
```bash
spack load python@3.10.7
```

* Import every Python module, object, and/or function that you need below.

In [1]:
import numpy as np
from numba import cuda, float32, float64
from __future__ import division
import math, time
import cupy as cp

<br>

### Exercise 1: Matrix Transpose

Before we examine matrix-matrix multiplication, we will first write a GPU kernel that transposes a square matrix.  This type of problem is a good introduction into how to use the CUDA threading model. The task for this exercise is to write a Numba CUDA kernel that will transpose a square matrix. 

**Before you start**, take a look at the following:
* Read over the following notebook that explanes Numba.CUDA kernels: [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb) 
* The first matrix-matrix multiplication code (the one that **doesn't** use shared memory) shown at [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). Understanding this code should give a pretty good idea on how to write the transpose kernel. The matrix-matrix kernel code from the NYU lab is shown below.
```python
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
```

**Tasks for this exercise**
* Write a Numba.CUDA kernel that transpose an $NxN$ square matrix.
* Be sure that the transpose kernel can transpose square matrices with sizes of $N$ as small as $N=2$ and as large as $N=10240$.
* Using shared memory is **not** required.


In [2]:
@cuda.jit
def transpose_kernel(A, B):
    row, col = cuda.grid(2)
    N = A.shape[0]
    if row < N and col < N:
        B[j, i] = A[i, j]

<br>

### Exercise 2: Using Numba CUDA to parallelize matrix multiplication: 

For this exercise, we will use Numba compiled GPU kernels that calculate matrix-matrix multiplication for square matrices. In particular, we will use a GPU kernel that doesn't used shared memory and compared to a GPU kernel that does use shared-memory. Please use the two kernel codes discussed in the following lab: [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). As you will see in this exercise, learning to use shared-memory (akin to user-controlled cache), can take a lot of practice, so in the next exercise, we examine how well the simple shared-memory kernel from the NYU lab compares to the optimized codes provided by NVIDIA in the CuPy package. 

#### The tasks for this exercise:
1. Copy the matrix-matrix kernel codes from the NYU lab. Test them for accurracy against *numpy.dot()* and also compare time runtimes these GPU kernels the numpy.dot() function as well. **Note:** Use [CUDA events](https://numba.readthedocs.io/en/stable/cuda-reference/host.html#events) when timing GPU kernel calls because the driver does not "block" the calling process (for case this is IPython). Insted, the kernel is sent to the GPU to run, and then the process (IPython) immediately continues to it's next bit of code. Contrary to GPU kernel calls, calls to copy data to or from the GPU will block the process. For these cases, the calls can be timed the same way that other Python calls are timed.<br> **For both GPU kernels:**
    - Test with square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 5120$, $N=10240$, and $N=20480$. **Tip**, first make sure you can get the GPU codes to work and that you get correct results by testing with $N_{test}=32$.
    - For each $N$ above, test the multiplication for both dtypes: *dtype=float32* and *dtype=float64*.
    - Calculate and show the error between your functions and the *numpy.dot()* function. 
    - Calculate and show the *speedup* (or *slowdown*) of your GPU kernel for each $N$ vs *numpy.dot()*. Be sure to include the array copy times in the "total-gpu-kernel runtime.
    - For each $N$ vs, calculate and show the *speedup* of your GPU kernel using *dtype=float32* vs *dtype=float64*. Be sure to include the array copy times in the "total-gpu-kernel runtime."
    
<br>

2. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
h_A = np.random.random((N, N)).astype(np.<float-type>)
h_B = np.random.random((N, N)).astype(np.<float-type>)
```    
<br>

3. For the device memory:
    - Create **d_A** and **d_B** by copying **h_A** and **h_B** to the GPU, and be sure to time the copies
    - Create **d_C** as device-array that is allocated on the GPU (device) only, and not on the host (**Do Not Copy**)
    
<br>

4. After the GPU matrix-matrix multiplication kernel finishes, **copy** the the *device-array* **d_C** to the *host-array* **h_C**, and be sure to time this copy.

<br>

5. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the speedup or slowdowns vs numpy as well as float32 vs float64. Remember, that your runtime for the GPU kernel include time to compile the kernel (not much you can do to control this). Futhermore, becasue you have to copy data to and off of the GPU, these copy times should be included in the "total-gpu-kernel runtime." 

In [3]:
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
        
# Host code

N = 10240

# Initialize the data arrays
A = np.random.random((N, N)).astype(np.float64)
B = np.random.random((N, N)).astype(np.float64)

# Start timing
start = cuda.event(timing=True)
start.record()

# Copy the arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)

# Allocate memory on the device for the result
C_global_mem = cuda.device_array((N,N))

# Configure the blocks
threadsperblock = (32, 32)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

# Copy the result back to the host
C = C_global_mem.copy_to_host()

end = cuda.event(timing=True)
end.record()
time = cuda.event_elapsed_time(start,end)

%time C_np = np.dot(A,B)
print('gpu time: ',time)

total_err = np.sum(np.abs(C-C_np))
print('total error: ',total_err)

CPU times: user 1min 2s, sys: 1.45 s, total: 1min 3s
Wall time: 8.47 s
gpu time:  65520.05078125
total error:  0.0006244602900551399


In [33]:
@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(32, 32), dtype=float64)
    sB = cuda.shared.array(shape=(32, 32), dtype=float64)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / 32)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * 32]
        sB[tx, ty] = B[tx + i * 32, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(32):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp
    
start_fast = cuda.event(timing=True)
start_fast.record()
    
# Copy the previously generated arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.device_array((N,N)) 

# Configure the blocks
threadsperblock = (32,32)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
C_fast = C_global_mem.copy_to_host()

end_fast = cuda.event(timing=True)
end_fast.record()

time_fast = cuda.event_elapsed_time(start_fast, end_fast)
print('gpu time: ',time_fast)

err_total_fast = np.sum(np.abs(C_fast-C_np))
print('total error: ',err_total_fast)

gpu time:  328905.59375
total error:  0.007062636951559398


1. N = 5120, dtype = float32:
numpy total time: 3.91 s<br>
"slow" gpu time:  5.686 s<br>
"slow" total error:  2546<br>
"fast" gpu time:  5.542 s<br>
"fast" total error:  2546<br>
2. N = 5120, dtype = float64:
numpy total time: 8.04 s<br>
"slow" gpu time:  7.091 s<br>
"slow" total error:  5.524e-05<br>
"fast" gpu time:  5.586 s<br>
"fast" total error:  5.524e-05<br>
3. N = 10240, dtype = float32:
numpy total time: 30.4 s<br>
"slow" gpu time:  47.589 s<br>
"slow" total error:  21912<br>
"fast" gpu time:  40.406 s<br>
"fast" total error:  21912<br>
4. N = 10240, dtype = float64:
numpy total time: 1min 3s<br>
"slow" gpu time:  66.436 s<br>
"slow" total error:  0.00062438<br>
"fast" gpu time:  40.319 s<br>
"fast" total error:  0.00062438<br>
5. N = 20480, dtype = float32:
numpy total time: 4min 8s<br>
"slow" gpu time:  464.977 s<br>
"slow" total error:  222547<br>
"fast" gpu time:  316.600 s<br>
"fast" total error:  222547<br>
6. N = 20480, dtype = float64:
numpy total time: 8min 2s<br>
"slow" gpu time:  837.106 s<br>
"slow" total error:  0.0070626<br>
"fast" gpu time:  328.905 s<br>
"fast" total error:  0.0070626<br><br>

It makes sense that the cuda code using shared memory would does not slow down when using float64 compared with float32, unlike the other methods. Also, the requirement to load the arrays to the gpu explains the differences between the numpy and gpu times, even though numpy.dot is much more optimized than even the fast kernel.

<br>

### Exercise 3: CuPy 

For this exercise, we will repeat what we did in *Exercise 2*. However, we will use *CuPy* functions, which are similar to *Numpy* funcstions with some added functions for copying data to-the-device-from-the-host and to-the-host-from-the-device. By using CuPy, we can depend on code that has been optimized for the GPU by NVIDIA, and instead of tyring to optimize our matrix-matrix multiplication kernels, we can use a built-in function to calculate the multiplication instead (i.e. [cupy.dot()](https://docs.cupy.dev/en/stable/reference/generated/cupy.dot.html#cupy.dot)).

**Tasks for this exercise:**
* Same as those listed in *Exercise 2*, but compare *cupy.dot()* to *numpy.dot()*.
* Also, reuse the host-arrays, *h_A* and *h_B* above. You will need to call the appropriate *CuPy* fuctions to copy these arrays to the GPU and to copy the result back to the host. You will **not** need to declare the deive-C array before calling *cupy.dot()* because the function will do it for you (like numpy does).


In [5]:
# Re-initialize arrays so I don't have to scroll a ton to change array parameters
N = 20480
A = np.random.random((N, N)).astype(np.float64)
B = np.random.random((N, N)).astype(np.float64)

# Set up timing stuff and calculate numpy result
%time C_np = np.dot(A,B)

start = cp.cuda.Event()
end = cp.cuda.Event()

start.record()

# Copy arrays to GPU
A_gpu = cp.asarray(A)
B_gpu = cp.asarray(B)

# Perform multiplication
C_gpu = cp.dot(A_gpu,B_gpu)

# Copy output back to host
C = cp.asnumpy(C_gpu)

end.record()

t_gpu = cp.cuda.get_elapsed_time(start,end)
print('gpu time: ',t_gpu)

total_err = np.sum(np.abs(C-C_np))
print('total error: ',total_err)

CPU times: user 8min 21s, sys: 18.3 s, total: 8min 39s
Wall time: 1min 8s
gpu time:  74919.796875
total error:  0.007062931569635111


1. N = 5120, dtype = float32:
numpy total time: 3.75 s<br>
gpu time:  424.4 ms<br>
total error:  29702.79<br>
2. N = 5120, dtype = float64:
numpy total time: 8.2 s<br>
gpu time:  1.753 s<br>
total error:  5.525e-5<br>
3. N = 10240, dtype = float32:
numpy total time: 29.6 s<br>
gpu time:  1.595 s<br>
total error:  335336<br>
4. N = 10240, dtype = float64:
numpy total time: 1 min 2 s<br>
gpu time:  10.796 s<br>
total error:  0.00062446<br>
5. N = 20480, dtype = float32:
numpy total time: 4 min 7 s<br>
gpu time:  7.27 s<br>
total error:  3796698<br>
4. N = 20480, dtype = float64:
numpy total time: 8min 39s<br>
gpu time:  74.92 s<br>
total error:  0.0070629<br>

<br>

### Exercise $\mathbf{\pi}$: CuPy Interoperability

Numba and CuPy device arrays (GPU arrays) can be accept each other's arrays. See [Interoperability](https://docs.cupy.dev/en/stable/user_guide/interoperability.html).

**Tasks for this exercise**
* Use the **device** arrays, **d_A** and **d_B**, that were created in *Exercise 2* to calculate the matrix-matrix multiplcation using *cupy.dot()*.
* Verify that you get the same results as you did in *Exercise 3*.
* You will need to "wrap" the device arrays before passing them to *cupy.dot()*. Read the *Interoperability* documentation linked above.
    - Time how long it takes (runtime) to "wrap" these arrays.
    - Compare this runtime to the runtime it took to create the device arrays in *Exercise 3*.
    - Provide a quick comment your thoughts on the runtime differences compared above.

In [10]:
start_cupy = cp.cuda.Event()
end_cupy = cp.cuda.Event()
start_cupy.record()

A_cupy = cp.asarray(A_global_mem)
B_cupy = cp.asarray(B_global_mem)

C_cupy = cp.dot(A_cupy,B_cupy)
C_host = cp.asnumpy(C_cupy)

end_cupy.record()
end_cupy.synchronize()
time = cp.cuda.get_elapsed_time(start_cupy,end_cupy)
print('cupy time: ',time)

C_numba = C_global_mem.copy_to_host()
total_err = np.sum(np.abs(C_host-C_numba))
print('total error: ',total_err)

cupy time:  9267.5341796875
total error:  0.0


The cupy function runs significantly faster, even with the wrapping and copying the result back to the host. Also, the results are the exact same. The speedup was ~6, which is rather significant. This is likely mostly due to optimization done automatically by cupy.